In [7]:
import numpy as np
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import os
import plotly.figure_factory as ff
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [3]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'bird'
))

In [ ]:
myquery = '''
SELECT band, lat_dd, lon_dd, event_year, event_month
FROM af_1
'''

df_1 = pd.read_sql_query(myquery, con = engine)

In [9]:
df_1['event_date'] = df_1['event_year'].astype(str) + '-' + df_1['event_month'].astype(str)
df_1['event_date'] = pd.to_datetime(df_1['event_date'], errors='coerce', format='%Y-%m')

In [10]:
timelist = df_1['event_date']

In [11]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [16]:
mymarkdown = '''
This is a dashboard we can see where the birds were observed
'''

In [18]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div(
    [
        # Stuff on top
        html.H1("Where are the birds"),
        html.H2("Data collected from North American Bird Banding Program"),
        html.H3("DS 6600: Data Engineering 1, UVA Data Science, Mia Yuan"),
        # Side bar
        html.Div([
            dcc.Markdown('Please select a time'),
            #Step 1: specify user input--from dropdown
            dcc.Dropdown(id = 'target_time', options = timelist, value = '1964-01'), 
            dcc.Markdown(mymarkdown)
        ], style = {'width': '24%', 'float':'left'}),
        # Main bar
        html.Div([
            dcc.Tabs([
                dcc.Tab(label = 'Geographic Info', children = [
                    #Stuff for the bio tab
                    dcc.Graph(id = 'geograph') #step 5: bring it to the place
                ]),
                # dcc.Tab(label = 'Bill Sponsorship', children = [
                #     #Stuff for the bill tab
                # ]),
                # dcc.Tab(label = 'Voting and Ideology', children = [
                #     #Stuff for the voting tab
                # ]),
                # dcc.Tab(label = 'Donors and Financial Info', children = [
                #     #Stuff for the donor tab
                # ]),
            ])
        ], style = {'width': '74%', 'float':'right'})
        
    ]
)
#Step 2,3: specify input and output; and a function
@app.callback([Output(component_id = 'geograph', component_property = 'figure')],
             [Input(component_id = 'target_time', component_property = 'value')])
def geograph(target_time):
    """
    Plot a heatmap of bird observations at a specific time on the world map.

    Parameters:
    - df: DataFrame containing bird observation data with 'LAT_DD' and 'LON_DD' columns
    - target_time: Target time in the format 'YYYY-MM' (e.g., '2022-11')
    """
    # Filter DataFrame for observations at the target time
    target_df = df_1[df_1['event_date'].dt.strftime('%Y-%m') == target_time]

    # Check if target_df is not empty before plotting
    if not target_df.empty:
        # Load world map shapefile
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

        # Create a GeoDataFrame from target_df
        gdf = gpd.GeoDataFrame(target_df, geometry=gpd.points_from_xy(target_df['LON_DD'], target_df['LAT_DD']))

        # Calculate bounding box based on min and max latitude/longitude
        minx, miny, maxx, maxy = gdf.total_bounds

        # Plot world map with calculated bounding box
        ax = world.cx[minx:maxx, miny:maxy].plot(figsize=(15, 10), color='white', edgecolor='black')

        # Plot bird observations as a transparent heatmap using seaborn kdeplot
        sns.kdeplot(x=gdf['LON_DD'], y=gdf['LAT_DD'], cmap='viridis', fill=True, ax=ax, alpha=0.5)

        # Plot original GeoDataFrame points on top of the heatmap
        gdf.plot(ax=ax, marker='o', color='red', markersize=5, alpha=0.8, label='Observation Points')

        # Set title and legend
        plt.title(f'Bird Observations Heatmap at {target_time}')
        plt.legend()

        # Show the plot
        plt.show()
    else:
        print(f'No bird observations found for {target_time}.')


if __name__== "__main__":
    app.run_server(mode= 'external', host = "127.0.0.0", debug=True)

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'